<a href="https://colab.research.google.com/github/Zozz98/Finance-Economics/blob/main/sandbox.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IMPORTS

In [1]:
import numpy as np
import itertools
import pandas as pd
import matplotlib.pyplot as plt

import plotly.express as px
import yfinance as yf
import pandas_datareader as pdr
import datetime as dt

from sklearn.cluster import KMeans, DBSCAN
from sklearn.metrics import silhouette_score

# GET DATA FROM WIKIPEDIA

In [2]:
url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
sp500_list = pd.read_html(url)
sp500_list = sp500_list[0]
sp500_list = sp500_list.drop(columns=['GICS Sub-Industry','Headquarters Location', 'Date added','CIK','Founded'], axis=1)
sp500_list.rename(columns={'GICS Sector':'Sector'},inplace=True)

# LOAD CLOSE PRICES FROM WIKIPEDIA DATA

In [3]:
end = dt.datetime.now()
start = end - dt.timedelta(days=252)

ticker_list = [i for i in sp500_list['Symbol']]

for i in range(len(ticker_list)):
    if ticker_list[i] == 'BRK.B':
        ticker_list[i] = 'BRK-B'
    elif ticker_list[i] == 'BF.B':
        ticker_list[i] = 'BF-B'

In [4]:
stocks = yf.download(ticker_list, start=start, end=end)['Close']

[*********************100%%**********************]  503 of 503 completed


# LOG RETURN AND SET COLUMNS

In [5]:
log_return = np.log(stocks / stocks.shift(1)).replace(np.nan, 0)

annual_return = log_return.mean() * 252
annual_volatility = log_return.std() * np.sqrt(252)

stock_data = pd.DataFrame()
stock_data['Symbol'] = ticker_list
stock_data['Security'] = sp500_list['Security']
stock_data['Sector'] = sp500_list['Sector']
stock_data['Return'] = list(round(annual_return, 3))
stock_data['Volatility'] = list(round(annual_volatility, 3))
stock_data['Ratio'] = list(round((annual_return / annual_volatility), 3))

In [6]:
log_return

Ticker,A,AAL,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,...,WTW,WY,WYNN,XEL,XOM,XYL,YUM,ZBH,ZBRA,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2023-08-23,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2023-08-24,0.003017,-0.014310,-0.026519,-0.003064,-0.026974,-0.013213,0.000133,-0.003813,-0.035052,-0.021586,...,0.011669,-0.016644,-0.006160,-0.008906,-0.007494,-0.009190,0.001081,-0.004625,-0.015560,-0.014707
2023-08-25,0.001589,0.000686,0.012564,0.000409,0.008543,0.004424,-0.010531,0.014600,0.024348,0.025410,...,0.002191,-0.004281,-0.000105,0.009949,0.017708,0.015138,0.001543,0.008015,0.009246,0.006916
2023-08-28,0.002003,0.010235,0.008807,0.005032,0.002897,-0.013720,-0.005779,0.006722,0.009213,0.012106,...,0.004368,0.007328,0.013627,0.004678,0.008371,0.006404,-0.003320,0.019677,0.005684,0.022500
2023-08-29,0.017360,0.011475,0.021576,0.001085,0.047183,0.010452,0.020809,0.007884,0.019898,0.012290,...,0.005457,0.004250,0.013138,0.001727,0.005937,0.004605,-0.003176,0.014361,0.013139,0.030496
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-04-24,-0.012360,-0.022026,0.012622,-0.010316,0.011674,-0.006527,-0.006311,-0.010438,0.008884,0.034007,...,0.003029,-0.005375,-0.017971,0.001085,0.000165,-0.000383,0.012132,-0.002884,0.003501,0.008787
2024-04-25,-0.008179,0.014974,0.005134,-0.003044,0.001043,-0.000281,-0.001503,-0.014586,-0.007743,0.007302,...,-0.029001,-0.002540,-0.004751,-0.005800,0.002310,-0.000459,-0.001764,-0.011788,0.052638,0.016303
2024-04-26,0.009996,-0.017851,-0.003479,-0.046933,0.007456,0.006250,-0.023377,-0.003209,0.008665,0.020155,...,-0.023511,-0.013118,-0.007481,-0.019272,-0.028169,0.013612,0.001482,-0.003346,0.015872,0.032462


# VOLATILITY-RETURN SCATTER PLOT

In [7]:
px.scatter(stock_data,
           x='Volatility',
           y='Return',
           hover_data=['Sector','Security'],
           hover_name='Symbol',
           title='S&P 500 stocks by Volatility/Return',
           color='Sector',
           width=750,
           height=500,
           template='plotly_dark'
           ).update_layout(title_x=0.5)

In [8]:
stock_data

,Symbol,Security,Sector,Return,Volatility,Ratio
0,MMM,3M,Industrials,0.204,0.267,0.764
1,AOS,A. O. Smith,Industrials,-0.131,0.388,-0.337
2,ABT,Abbott,Health Care,-0.089,0.201,-0.444
3,ABBV,AbbVie,Health Care,0.146,0.183,0.802
4,ACN,Accenture,Information Technology,0.310,0.347,0.895
...,...,...,...,...,...,...
498,XYL,Xylem Inc.,Industrials,0.382,0.194,1.970
499,YUM,Yum! Brands,Consumer Discretionary,0.128,0.153,0.834
500,ZBRA,Zebra Technologies,Information Technology,0.067,0.215,0.313
501,ZBH,Zimmer Biomet,Health Care,0.222,0.383,0.579


# FIND NUMBER OF CLUSTERS WITH ELBOW-METHOD

In [9]:
X = stock_data[['Return', 'Volatility']]
inertia_list = []

for k in range(1,16):
    kmeans = KMeans(n_clusters=k, n_init='auto')
    kmeans.fit(X)
    inertia_list.append(kmeans.inertia_)

px.line(inertia_list,
        title='Elbow Curve',
        width=750,
        height=500,
        labels={
            "index":"Number of Clusters",
            "value":"Sum of Squared Error"
        },
        template='plotly_dark'
        ).update_layout(showlegend=False, title_x=0.5)

# KMEANS MODEL FIT

In [14]:
kmeans = KMeans(n_clusters=2, n_init='auto').fit(X)
labels = kmeans.labels_
stock_data['KMeansLabel'] = labels

# PLOT STOCKS BY CLUSTERS

In [15]:
px.scatter(X,
           x='Volatility',
           y='Return',
           color=labels,
           title='KMeansLabel',
           hover_name=stock_data['Symbol'],
           width=750,
           height=500,
           template='plotly_dark'
           ).update_layout(title_x=0.5)

In [12]:
silhouette = []

for k in range(2,16):
    kmeans = KMeans(n_clusters=k, n_init='auto')
    kmeans.fit(X)
    silhouette.append(silhouette_score(X, kmeans.labels_))

In [13]:
px.line(silhouette,
        title='Silhouette Curve',
        width=750,
        height=500,
        labels={
            "index":"Number of Clusters",
            "value":"Sum of Squared Error"
        },
        template='plotly_dark'
        ).update_layout(showlegend=False, title_x=0.5)

In [19]:
dividends = []

for i in ticker_list:
    info = yf.Ticker(i).info
    div = info.get('dividendYield')
    dividends.append(div)

In [33]:
print(yf.Ticker('MMM').info.get('exDividendDate'))
print(pd.to_datetime(1707955200, unit='s'))
print(dt.datetime.fromtimestamp(1707955200))

1707955200
2024-02-15 00:00:00
2024-02-15 00:00:00


In [34]:
for (i,j) in zip(ticker_list, dividends):
    print(f'Ticker: {i}, DividendRate: {j}')

Ticker: MMM, DividendRate: 0.0626
Ticker: AOS, DividendRate: 0.0155
Ticker: ABT, DividendRate: 0.0208
Ticker: ABBV, DividendRate: 0.0381
Ticker: ACN, DividendRate: 0.0171
Ticker: ADBE, DividendRate: None
Ticker: AMD, DividendRate: None
Ticker: AES, DividendRate: 0.0378
Ticker: AFL, DividendRate: 0.0239
Ticker: A, DividendRate: 0.0069
Ticker: APD, DividendRate: 0.0295
Ticker: ABNB, DividendRate: None
Ticker: AKAM, DividendRate: None
Ticker: ALB, DividendRate: 0.013300001
Ticker: ARE, DividendRate: 0.0438
Ticker: ALGN, DividendRate: None
Ticker: ALLE, DividendRate: 0.015800001
Ticker: LNT, DividendRate: 0.038599998
Ticker: ALL, DividendRate: 0.0216
Ticker: GOOGL, DividendRate: 0.0049
Ticker: GOOG, DividendRate: 0.0049
Ticker: MO, DividendRate: 0.089499995
Ticker: AMZN, DividendRate: None
Ticker: AMCR, DividendRate: 0.0559
Ticker: AEE, DividendRate: 0.0363
Ticker: AAL, DividendRate: None
Ticker: AEP, DividendRate: 0.040900003
Ticker: AXP, DividendRate: 0.012
Ticker: AIG, DividendRate: 0.0

In [35]:
stock_data['Dividend'] = dividends

In [40]:
no_dividend_shares = stock_data.loc[stock_data['Dividend'] == 0].count()
dividend_shares = stock_data.loc[stock_data['Dividend'] > 0].count()

print(f'Number of shares does not have Dividend: {no_dividend_shares[1]}')
print(f'Number of shares does have Dividend: {dividend_shares[1]}')

Number of shares does not have Dividend: 0
Number of shares does have Dividend: 406


In [41]:
stock_data_dividend = stock_data.loc[stock_data['Dividend'] > 0]

In [42]:
px.scatter(stock_data_dividend,
           x='Volatility',
           y='Return',
           hover_data=['Sector'],
           hover_name='Symbol',
           title='S&P 500 dividend stocks by Volatility/Return',
           color='Ratio',
           width=750,
           height=500,
           template='plotly_dark'
           ).update_layout(title_x=0.5)

In [52]:
epsilons = np.linspace(0.01, 10, 50)
min_samples = np.arange(2,30, 3)

combinations = list(itertools.product(epsilons, min_samples))

N = len(combinations)

In [58]:
def get_scores_and_labels(combinations, X):
    scores = []
    all_labels = []

    for i, (eps, num_samples) in enumerate(combinations):
        dbscan_model = DBSCAN(eps=eps, min_samples=num_samples).fit(X)
        labels = dbscan_model.labels_
        labels_set = set(labels)
        num_clusters = len(labels_set)

        if -1 in labels_set:
            num_clusters -= 1

        if (num_clusters < 2) or (num_clusters > 50):
            scores.append(-10)
            all_labels.append('Bad')
            c = (eps, num_samples)
            print(f'Combination {c} on iter {i+1} of {N} has {num_clusters} clusters')
            continue

        scores.append(silhouette_score(X, labels))
        all_labels.append(labels)
        print(f'Index: {i}, Score: {scores[-1]}, Labels: {all_labels[-1]}')

    best_index = np.argmax(scores)
    best_parameters = combinations[best_index]
    best_labels = all_labels[best_index]
    best_score = scores[best_index]

    return {
        'best_epsilon':best_parameters[0],
        'best_min_samples':best_parameters[1],
        'best_labels':best_labels,
        'best_score':best_score
    }

best_dict = get_scores_and_labels(combinations, X)

Combination (0.01, 2) on iter 1 of 500 has 78 clusters
Index: 1, Score: -0.4885873521472572, Labels: [-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  3 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  5 -1 -1 -1 -1 -1 -1  0 -1 -1 -1 -1
 -1  5 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  1 -1  4 -1 -1 -1
 -1 -1 -1 -1 -1  6 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  1
 -1 -1 -1 -1 -1 -1  6  7 -1 -1 -1 -1 -1 -1 -1 -1 -1  6 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1  2 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
  7 -1 -1  0 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  2 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  5 -1 -1 -1 -1  0 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  3 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1  2 -1 -1 -1 -1 -1 -1 -1 -1 -1  0 -1 -1 -1  3 -1 -1 -1
  2 -1 -1 -1 -1 -1 -1  4  4 -1 -1  5 -1 -1 -1 -1 -1  6 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1  3 -1 -

In [59]:
best_dict

{'best_epsilon': 0.01,
 'best_min_samples': 5,
 'best_labels': array([-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1,  3, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1,  5, -1, -1, -1, -1, -1, -1,  0, -1, -1, -1, -1, -1,  5, -1,
        -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,  1, -1,
         4, -1, -1, -1, -1, -1, -1, -1, -1,  6, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,  1, -1, -1, -1, -1, -1, -1,
         6,  7, -1, -1, -1, -1, -1, -1, -1, -1, -1,  6, -1, -1, -1, -1, -1,
        -1, -1, -1, -1, -1, -1, -1,  2, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1, -1, -1, -1, -1, -1,  7, -1, -1,  0, -1, -1, -1, -1, -1,
        -1, -1, -1, -1, -1,  2, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,  5, -1, -1, -1, -1,
         0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1,  3, -1, -1, -1

In [60]:
stock_data['DBSCANLabel'] = best_dict['best_labels']

In [63]:
px.scatter(stock_data,
           x='Volatility',
           y='Return',
           hover_data=['Sector'],
           hover_name='Symbol',
           title='S&P 500 dividend stocks by Volatility/Return',
           color='DBSCANLabel',
           width=750,
           height=500,
           template='plotly_dark'
           ).update_layout(title_x=0.5)

In [64]:
px.scatter(stock_data_dividend,
           x='Volatility',
           y='Return',
           hover_data=['Sector'],
           hover_name='Symbol',
           title='S&P 500 dividend stocks by Volatility/Return',
           color='Ratio',
           width=750,
           height=500,
           template='plotly_dark'
           ).update_layout(title_x=0.5)

In [74]:
X_dividend = stock_data_dividend[['Volatility','Return']]

In [75]:
best_dict = get_scores_and_labels(combinations, X_dividend)

Combination (0.01, 2) on iter 1 of 500 has 65 clusters
Index: 1, Score: -0.41740420383390126, Labels: [-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  0 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1  1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  2 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  0 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  0 -1 -1 -1 -1 -1 -1 -1 -1 -1  1  1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  0 -1 -1 -1 -1 -1 -1
 -1  2 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 

In [76]:
stock_data_dividend['DBSCANLabel'] = best_dict['best_labels']

<ipython-input-76-2cdf091ff815>:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [77]:
px.scatter(stock_data_dividend,
           x='Volatility',
           y='Return',
           hover_data=['Sector'],
           hover_name='Symbol',
           title='S&P 500 dividend stocks by Volatility/Return',
           color='DBSCANLabel',
           width=750,
           height=500,
           template='plotly_dark'
           ).update_layout(title_x=0.5)